In [1]:
##############################################################################################################
####
####   Mask Effluents
####   By Cascade Tuholske June 2020
####
####   This script masks inland watersheds from the 
####   effluent rasters and makes float64 tif
####   files to run the zonal stats on
####
##############################################################################################################

In [2]:
#### Dependencies 
##############################################################################################################
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt 

In [3]:
#### File Paths and FN
# effluent rsts
DATA_IN = '/home/cascade/projects/wastewater/data/'
MASK_FN = DATA_IN+'interim/inlandwatersheds_mask.tif'
effluent_rsts = [DATA_IN+'interim/effluent_N.tif', DATA_IN+'interim/effluent_N_treated.tif', 
                 DATA_IN+'interim/effluent_N_septic.tif', DATA_IN+'interim/effluent_N_open.tif']


In [4]:
def mask_effluent(effluent_fn, mask_fn, out_fn):
    
    """Function opens effluent raster and masks inland watershed pixels\
    Args:
        effluent_fn = effluent raster fn
        mask_fn = mask raster fn
        out_fn = file name out
    """
    
    # Open effluent raster
    rst = rasterio.open(effluent_fn)
    meta = rst.meta
    band = rst.read(1)
    
    # mask inland watersheds
    mask = rasterio.open(mask_fn).read(1)
    mask[mask == 1] = 2 # revalue mask so inland watersheds are = 0 
    mask[mask == 0] = 1
    mask[mask == 2] = 0
    
    band_out = band * mask 
    
    print(band_out.max())

    #Save new data type and mask out
    with rasterio.open(out_fn, 'w', **meta) as dst:
        dst.write(band_out, 1)
    
    print('Done', out_fn)

In [5]:
# effluent rsts
effluent_rsts = [DATA_IN+'interim/effluent_N.tif', DATA_IN+'interim/effluent_N_treated.tif', 
                 DATA_IN+'interim/effluent_N_septic.tif', DATA_IN+'interim/effluent_N_open.tif']

# Make masked rasters
for rst in effluent_rsts:
    
    # Get data type
    data = rst.split('interim/')[1].split('.')[0]
    print(data)

    # Raster Mask 64 
    print('Starting mask', rst)
    rst_out = DATA_IN+'interim/'+data+'_mask.tif'
    mask_effluent(rst, MASK_FN, rst_out)
    
    print('\n')



effluent_N
Starting mask /home/cascade/projects/wastewater/data/interim/effluent_N.tif
1271872000.0
Done /home/cascade/projects/wastewater/data/interim/effluent_N_mask.tif


effluent_N_treated
Starting mask /home/cascade/projects/wastewater/data/interim/effluent_N_treated.tif
176486930.0
Done /home/cascade/projects/wastewater/data/interim/effluent_N_treated_mask.tif


effluent_N_septic
Starting mask /home/cascade/projects/wastewater/data/interim/effluent_N_septic.tif
89186190.0
Done /home/cascade/projects/wastewater/data/interim/effluent_N_septic_mask.tif


effluent_N_open
Starting mask /home/cascade/projects/wastewater/data/interim/effluent_N_open.tif
1263654800.0
Done /home/cascade/projects/wastewater/data/interim/effluent_N_open_mask.tif




In [9]:
from glob import glob

# Open watershed basins polygons and stack em
basins_dir = glob(DATA_IN+'interim/basins_crs/*59004.shp')

# empty df to stack all the watershed polys
columns= (['ID','GRIDCODE','inspect','area','PNTPOLYCNT','basin_id','MWa_in_km2','geometry'])
watersheds = pd.DataFrame(columns = columns)

# Open watershed polys
for shp_fn in basins_dir:
    basins = pd.DataFrame(gpd.read_file(shp_fn))
    watersheds = watersheds.append(basins, sort = False)

# watersheds = gpd.GeoDataFrame(watersheds) # to geo data frame
# watersheds.crs = countries.crs # Set crs

# Drop inland watersheds w/ pourpoints file
inland_pp_fn = DATA_IN+'interim/pourpoints_inland.shp'
inland_pp = gpd.read_file(inland_pp_fn)
inland_pp = inland_pp[inland_pp['land-ocean'] == 0] # drop inland watersheds 
inland_pp = inland_pp['basin_id']
watersheds = watersheds.merge(inland_pp, on = 'basin_id', how = 'inner') # merge in

# Drop inland watersheds w/ pourpoints file
inland_pp_fn = DATA_IN+'interim/pourpoints_inland.shp'
inland_pp = gpd.read_file(inland_pp_fn)
inland_pp = inland_pp[inland_pp['land-ocean'] == 0] # drop inland watersheds 
inland_pp = inland_pp['basin_id']
watersheds = watersheds.merge(inland_pp, on = 'basin_id', how = 'inner') # merge in

In [13]:
watersheds = gpd.GeoDataFrame(watersheds) # to geo data frame

fn_out = DATA_IN+'interim/watersheds_coastal.shp'
watersheds.to_file(fn_out)
